# Testing Data Prep Kit Transforms

This notebook verifies the VLLM Transforms. 
**Note:** We use `%%writefile` to ensure the transform code exists in the environment under `dpk_extn`, avoiding git sync issues.

In [3]:
# 1. Install Dependencies
!pip install -q vllm ray[default] data-prep-toolkit-transforms jinja2 pandas pyarrow huggingface_hub

In [4]:
# 2. Setup Directory Structure
import os
os.makedirs("dpk_extn", exist_ok=True)
print("Created 'dpk_extn' directory.")

Created 'dpk_extn' directory.


In [ ]:
from dpk_extn.vllm_transform import VLLMTransform, VLLMTransformConfig

In [ ]:
# 3. Verify Import
import sys
import os
# Ensure current directory is in path (to import 'dpk_extn')
print(sys.path)
print(os.getcwd())
print('----------------')
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())
   
try:
    from dpk_extn.vllm_transform import VLLMTransform
    print("✅ Import successful from dynamically written files!")
except ImportError as e:
    print(f"❌ Import failed: {e}")


['/content', '/env/python', '/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/usr/local/lib/python3.12/dist-packages', '/usr/local/lib/python3.12/dist-packages/nvidia_cutlass_dsl/python_packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.12/dist-packages/IPython/extensions', '/root/.ipython']
/content
❌ Import failed: No module named 'dpk_extn.vllm_transform'


In [ ]:
# 4. Run Inference
import pyarrow as pa

data = [{"instruction": "What is the capital of France?", "id": 1}]
table = pa.Table.from_pylist(data)

config = {
    "model": "Qwen/Qwen2.5-1.5B-Instruct", # Small model for tests
    "tensor_parallel_size": 1,
    "batch_size": 10, 
    "map_column": "instruction"
}

print("Initializing & Running...")
try:
    transform = VLLMTransform(config)
    result = transform.transform(table)
    print("Result:", result.to_pandas()["completions"].tolist())
except Exception as e:
    print(f"Execution failed (expected if no GPU/dependencies): {e}")